In [1]:
import pandas as pd

customer_path = 'dataset/customer.csv'
sales_path = 'dataset/sales.csv'
goods_path = 'dataset/goods.csv'
category_path = 'dataset/x_tree.csv'
publisher_path = 'dataset/x_tag.csv'

customer_df = pd.read_csv(customer_path)
sales_df = pd.read_csv(sales_path)
goods_df = pd.read_csv(goods_path)
category_df = pd.read_csv(category_path)
publisher_df = pd.read_csv(publisher_path)

sales_df.head()

,invoice_id,register_date,member_id,province_id,city_id,good_id
0,2,2014-05-11 08:31:37,5,4.0,72.0,15347
1,5,2014-05-12 15:16:06,8,8.0,110.0,2833
2,5,2014-05-12 15:16:06,8,8.0,110.0,8714
3,7,2014-05-12 22:23:19,9,27.0,337.0,15672
4,11,2014-05-16 11:55:43,12,8.0,121.0,15001


In [2]:
factorize_member_id = pd.factorize(sales_df.member_id)[0]
sales_df['factorize_member_id'] = factorize_member_id

factorize_good_id = pd.factorize(sales_df.good_id)[0]
sales_df['factorize_good_id'] = factorize_good_id

sales_df.head()

,invoice_id,register_date,member_id,province_id,city_id,good_id,factorize_member_id,factorize_good_id
0,2,2014-05-11 08:31:37,5,4.0,72.0,15347,0,0
1,5,2014-05-12 15:16:06,8,8.0,110.0,2833,1,1
2,5,2014-05-12 15:16:06,8,8.0,110.0,8714,1,2
3,7,2014-05-12 22:23:19,9,27.0,337.0,15672,2,3
4,11,2014-05-16 11:55:43,12,8.0,121.0,15001,3,4


In [3]:
unique_data_df = sales_df.drop_duplicates()



In [4]:
factorizeMemberId2MemberId = dict(zip(sales_df.factorize_member_id, sales_df.member_id))
memberId2FactorizeMemberId = dict(zip(sales_df.member_id, sales_df.factorize_member_id))

factorizeGoodsId2GoodsId = dict(zip(sales_df.factorize_good_id, sales_df.good_id))
goodsId2FactorizeGoodsId = dict(zip(sales_df.good_id, factorize_good_id))

In [5]:
adjacency = unique_data_df[['factorize_member_id', 'factorize_good_id']]
adjacency.head(10)

,factorize_member_id,factorize_good_id
0,0,0
1,1,1
2,1,2
3,2,3
4,3,4
5,3,5
6,3,6
7,3,7
8,4,8
9,4,9


In [6]:
good_category_feature = goods_df['catgory'].str.get_dummies('|').values

In [7]:
category = goods_df.catgory.str.split('|').apply(lambda x: list(map(int, x)) if isinstance(x, list) else []).tolist()
category[:10]

[[153],
 [66],
 [139, 141],
 [118],
 [139, 140],
 [130],
 [120],
 [167],
 [64, 73],
 [62, 65, 66]]

In [8]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
category_encoded = pd.DataFrame(mlb.fit_transform(category), columns=mlb.classes_, index = goods_df.index)

In [9]:
good_id_to_category_df = pd.concat([goods_df['good_id'], category_encoded], axis=1)
good_id_to_category_df

,good_id,29,31,32,35,37,38,40,41,42,...,170,171,173,178,189,190,191,192,193,197
0,162,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,163,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,165,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,166,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,167,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57088,105771,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
57089,105772,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
57090,105773,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
57091,105774,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
from sentence_transformers import SentenceTransformer
import torch
import numpy as np

In [11]:
category_df

,node_id,node_title
0,141,آثار و پیامدهای صهیونیسم و استکبار بین المللی ...
1,136,آثار و پیامدهای علم و تکنولوژی (3-13)
2,91,آداب معاشرت، دوستی و محبت (7-28)
3,169,آداب و رسوم (8-28)
4,88,آشپزی و تغذیه (4-28)
...,...,...
157,101,کتاب های درسی (2-25)
158,98,کتاب های درسی (2-26)
159,102,کتاب های کمک درسی (3-25)
160,99,کتاب های کمک درسی (3-26)


In [12]:
publisher_df.size

4206

In [13]:
publisher_df.dropna()

,tag_id,tag_word
5,1937,& پخش فرهنگ - کتاب
6,9607,&انتشارات دلیل ما - ناشر
7,9608,&انتشارات دلیل ما - کتاب
8,9113,&انتشارات لیله القدر - ناشر
9,9114,&انتشارات لیله القدر - کتاب
...,...,...
2098,971,یادگیری
2099,1582,یهود در قرآن
2100,897,یهودیت
2101,1652,یهودیت - اعتقادات


In [14]:
publisher_df[publisher_df.tag_word.apply(lambda x:type(x) not in [str])]

,tag_id,tag_word
0,26544,NaN
1,28232,NaN
2,28233,NaN
3,29070,NaN
4,84193,NaN


In [17]:
bert_model = SentenceTransformer('all-MiniLM-L6-v2')
while(torch.no_grad()):
    tags = bert_model.encode(category_df.node_title.to_list(), convert_to_tensor=True, show_progress_bar=True)
    tags = tags.cpu()

tags

Batches:   0%|          | 0/6 [00:00<?, ?it/s]

Batches:   0%|          | 0/6 [00:00<?, ?it/s]

Batches:   0%|          | 0/6 [00:00<?, ?it/s]

Batches:   0%|          | 0/6 [00:00<?, ?it/s]

Batches:   0%|          | 0/6 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/6 [00:00<?, ?it/s]

Batches:   0%|          | 0/6 [00:00<?, ?it/s]

Batches:   0%|          | 0/6 [00:00<?, ?it/s]

Batches:   0%|          | 0/6 [00:00<?, ?it/s]

KeyboardInterrupt: 